In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from collections import OrderedDict
import torch.optim as optim
import time
from src import *
import math
from tqdm import tqdm
from tqdm.notebook import tqdm

In [2]:
# parameters
ϵ = 8 / 256
K = 5
retrain = 10
epoch_count = 10
batch_size = 10
pre_train = False

In [3]:
# FASION-MNIST INPUT
transform = transforms.Compose(
    [transforms.RandomCrop(32, padding=4),
     transforms.ToTensor(),
     transforms.RandomHorizontalFlip(p=0.5),
    ])

transform_test = transforms.Compose(
    [transforms.RandomCrop(32, padding=4),
     transforms.ToTensor(),
     transforms.RandomHorizontalFlip(p=0.5),
    ])

trainset = torchvision.datasets.FashionMNIST(root='./data', train=True,  transform= transform, download = True )
# trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

def trainloadicator(b):
    return torch.utils.data.DataLoader(trainset, 
        batch_size=b,
        shuffle=True, num_workers=4, 
        pin_memory=True, drop_last=True)
trainloader = trainloadicator(batch_size)

testset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform_test)
# testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,   shuffle=False, num_workers=4)

dataiter = iter(trainloader)
images, labels = dataiter.next()

In [4]:
norm = StandardScalerLayer(lambda: map(lambda x: x[0], trainloader))

criterion = nn.CrossEntropyLoss()

def mkmodel(ϵ=ϵ):
    model = WideResNet(28, 10, 10, 0.1)
    adv = AdversarialForFree(ϵ, 0, 1)
    return nn.Sequential(OrderedDict([
        ('adv', adv),
        ('normalizer', norm),
        ('resnet', model)])).cuda()

imgsize = images.size()[1:]
imgsize

torch.Size([1, 32, 32])

In [20]:
train_loader = None
train_batch = None

model = mkmodel()

# print(model)

optimizer = optim.Adam( model.parameters()  , weight_decay=5e-4)
# torch.load('model1.pth') 

for epoch in range(math.ceil(epoch_count / K)):  # loop over the dataset multiple times
                   
    b = 10
    if train_batch != b:
        print(f'setting batch size to {b}')
        train_loader = trainloadicator(b)
        train_batch = b

    logs = Logisticator()
    model.train()
                   
    for i, data in tqdm(enumerate(train_loader, 0) ):
        # get the inputs; data is a list of [inputs, labels]

        inputs, labels = map(lambda x: x.cuda(), data)

        for k in range(K):
            # zero the parameter gradients
            optimizer.zero_grad()
            # forward + backward + optimize
            outputs = model(inputs)
            loss = F.cross_entropy(outputs, labels)
            loss.backward()
            optimizer.step()
            model.adv.step()

            acc = accuracy(outputs, labels)
            logs.add(acc, loss.item(), inputs.size(0))
    
    print(f'train \t {epoch + 1}: {logs}')
    
    model.train(False)
    
    # valdiation loss
    with torch.no_grad():
        logs = Logisticator()
        for data in tqdm(testloader):
            inputs, labels = map(lambda x: x.cuda(), data)
            outputs = model(inputs)
            loss = F.cross_entropy(outputs, labels)

            acc = accuracy(outputs, labels)
            logs.add(acc, loss.item(), inputs.size(0))
            
        print(f'Normal \t {epoch + 1}: {logs}')
    
    # FGSM loss
    logs = Logisticator()
    for data in tqdm(testloader):

        inputs, labels = map(lambda x: x.cuda(), data)
        noise = FGSM(model, lambda x: F.cross_entropy(x, labels), inputs, ϵ, K)
        
        with torch.no_grad():
            outputs = model(inputs + noise)
            loss = F.cross_entropy(outputs, labels)
            acc = accuracy(outputs, labels)
            logs.add(acc, loss.item(), inputs.size(0))
    print(f'FGSM \t {epoch + 1}: {logs}')
    
    # PGD loss
    logs = Logisticator()
    for data in tqdm(testloader):

        inputs, labels = map(lambda x: x.cuda(), data)
        noise = PGK(model, lambda x: F.cross_entropy(x, labels), inputs, ϵ, K)
        
        with torch.no_grad():
            outputs = model(inputs + noise)
            loss = F.cross_entropy(outputs, labels)
            acc = accuracy(outputs, labels)
            logs.add(acc, loss.item(), inputs.size(0))
    print(f'PGK \t {epoch + 1}: {logs}')
    
print('Finished Training')


setting batch size to 10


0it [00:00, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fe1686cc310>
Traceback (most recent call last):
  File "/home/mheuillet/anaconda3/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/home/mheuillet/anaconda3/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/home/mheuillet/anaconda3/lib/python3.9/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fe1686cc310>
Traceback (most recent call last):
  File "/home/mheuillet/anaconda3/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/home/mheuillet/anaconda3/lib/python3.9/site-packages/torch/utils/data/dataloader

train 	 1: 0.8650 67.1% 5620.4s


  0%|          | 0/1000 [00:00<?, ?it/s]

...

Normal 	 2: 0.5296 80.1% 63.0s


  0%|          | 0/1000 [00:00<?, ?it/s]

FGSM 	 2: 0.7006 73.3% 228.1s


  0%|          | 0/1000 [00:00<?, ?it/s]

PGK 	 2: 0.7122 73.0% 886.7s
Finished Training


In [6]:
train_loader = None
train_batch = None

K = 1
model = mkmodel()
optimizer = optim.Adam(model.parameters(), weight_decay=5e-4)

for epoch in tqdm( range(math.ceil(epoch_count / K)) ):  # loop over the dataset multiple times
                   
    b = 10
    if train_batch != b:
        print(f'setting batch size to {b}')
        train_loader = trainloadicator(b)
        train_batch = b

    logs = Logisticator()
    model.train()
                   
    for i, data in tqdm( enumerate(train_loader, 0) ):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = map(lambda x: x.cuda(), data)
        for k in range(K) :
            # zero the parameter gradients
            optimizer.zero_grad()
                
            # forward + backward + optimize
            outputs = model(inputs)
            loss = F.cross_entropy(outputs, labels)
            loss.backward()
            
            optimizer.step()
            model.adv.step()
                     
            acc = accuracy(outputs, labels)
            logs.add(acc, loss.item(), inputs.size(0))
    
    print(f'train \t {epoch + 1}: {logs}')
    
    
    
    model.train(False)
    
    
    # valdiation loss
    with torch.no_grad():
        logs = Logisticator()
        for data in tqdm( testloader ):
            inputs, labels = map(lambda x: x.cuda(), data)
            outputs = model(inputs)
            loss = F.cross_entropy(outputs, labels)

            acc = accuracy(outputs, labels)
            logs.add(acc, loss.item(), inputs.size(0))
            
        print(f'val \t {epoch + 1}: {logs}')
    
    # adv loss
    logs = Logisticator()
    for data in tqdm( testloader ):
        inputs, labels = map(lambda x: x.cuda(), data)
        noise = FGSM(model, lambda x: F.cross_entropy(x, labels), inputs, ϵ, K)
        
        with torch.no_grad():
            outputs = model(inputs + noise)
            loss = F.cross_entropy(outputs, labels)

            acc = accuracy(outputs, labels)
            logs.add(acc, loss.item(), inputs.size(0))
    print(f'FGSM \t {epoch + 1}: {logs}')
    
    
    # adv loss
    logs = Logisticator()
    for data in tqdm( testloader ):
        inputs, labels = map(lambda x: x.cuda(), data)
        noise = PGK(model, lambda x: F.cross_entropy(x, labels), inputs, ϵ, K)
        
        with torch.no_grad():
            outputs = model(inputs + noise)
            loss = F.cross_entropy(outputs, labels)

            acc = accuracy(outputs, labels)
            logs.add(acc, loss.item(), inputs.size(0))
    print(f'PGK \t {epoch + 1}: {logs}')
    
print('Finished Training')
del model
torch.cuda.empty_cache() 

  0%|          | 0/10 [00:00<?, ?it/s]

setting batch size to 10


0it [00:00, ?it/s]

train 	 1: 0.7551 72.5% 1156.2s


  0%|          | 0/1000 [00:00<?, ?it/s]

...

val 	 10: 0.2828 89.4% 63.9s


  0%|          | 0/1000 [00:00<?, ?it/s]

FGSM 	 10: 1.7406 46.2% 229.0s


  0%|          | 0/1000 [00:00<?, ?it/s]

PGK 	 10: 1.7344 46.3% 230.7s
Finished Training
